In [ ]:
!pip install langchain
!pip install openai
!pip install chromadb
!pip install azure-storage-blob
!pip install tiktoken


In [ ]:
# !pip install --index-url=https://pkgs.dev.azure.com/azure-sdk/public/_packaging/azure-sdk-for-python/pypi/simple/ azure-search-documents==11.4.0a20230509004
# !pip install azure-identity

In [ ]:
import os
from langchain.retrievers import AzureCognitiveSearchRetriever
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import AzureOpenAI
from langchain.chains import RetrievalQA

In [ ]:
os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = "acs-openai-test-3"
os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] ="azureblob-index-17files"
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = ""


In [ ]:
query = input("Enter your query: ")

Enter your query: Northwind Standard Health


In [ ]:
retriever = AzureCognitiveSearchRetriever(content_key="content")
docs = retriever.get_relevant_documents(query)


In [ ]:
docs

In [ ]:
#document= Document(page_content)
page_content = docs[0].page_content

file_path = docs[0].metadata['metadata_storage_name']

print(file_path)

Northwind_Standard_Benefits_Details.pdf


In [ ]:
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://testnew.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
#'''from langchain.llms import OpenAI
#qa = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0, max_tokens=3000), chain_type="stuff", retriever=retriever)
#query = "cause of homelessness"
#qa.run(query)'''

from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

qa = RetrievalQA.from_chain_type(AzureChatOpenAI(deployment_name="testgptturbo",model_name="gpt-35-turbo-16k", temperature=0, max_tokens=3000), chain_type="map_reduce", retriever=retriever)
#qa = RetrievalQAWithSourcesChain.from_chain_type(AzureChatOpenAI(deployment_name="test1",model_name="gpt-35-turbo", temperature=0, max_tokens=3000), chain_type="map_reduce", retriever=retriever, reduce_k_below_max_tokens=True)
#query = "Where does homeless person stay?"
qa.run(query)

#qa({"question": qa}, return_only_outputs=True)



InvalidRequestError: ignored

In [ ]:
import openai

response = openai.ChatCompletion.create(
  engine="test1",
  messages = [{"role":"system","content":"You are an AI assistant that helps people find information. Limit tokens to 2000."},{"role": "user", "content": query}],
  temperature=0,
  max_tokens=800,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)

answer = response.choices[0].message.content
print("Answer:", answer)

Answer: The Northwind Standard Health plan is a health insurance plan offered by Northwind Insurance Services. It provides coverage for a range of medical services, including doctor visits, hospital stays, prescription drugs, and preventive care. The plan also includes coverage for mental health and substance abuse treatment. The cost of the plan varies depending on factors such as age, location, and health status. To learn more about the Northwind Standard Health plan, you can visit the Northwind Insurance Services website or contact their customer service department.


In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
#don't run
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings(openai_api_version='2020-11-07')
vectordb = Chroma.from_documents(documents=docs, embedding=embeddings)

InvalidRequestError: ignored

In [ ]:
#don't run
from langchain.llms import OpenAI

chain = RetrievalQA.from_chain_type(OpenAI(), retriever = vectordb.as_retriever(), chain_type="stuff")
query = "Where does homeless person stays?"
chain.run(query)

InvalidRequestError: ignored